# Preparing Linkset With DBPEDIA




## Getting a cleaned linkset

In [46]:
import pandas as pd
### Global variables 
path="data/" # path where to find data
namesa=['sBT','sprefLabel','s','o','oprefLabel','oBT', 'KindOfLink'] #column names for training data frame

# reading Thist2DBPEDIA_ok.csv
THIST2DBPEDIA_df=pd.read_csv(path+'Thist2DBPEDIA_ok.csv',delimiter=";",names=namesa)
#deleting row 0 which contains the old colums name
THIST2DBPEDIA_df=THIST2DBPEDIA_df.drop([0])
#THIST2DBPEDIA_df.head()
THIST2DBPEDIA_df_linkeset=THIST2DBPEDIA_df[['s','o','KindOfLink']].drop_duplicates()
THIST2DBPEDIA_df_linkeset.head()


,s,o,KindOfLink
1,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Madagascar,E
12,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Niger,E
16,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Nubia,E
20,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Sahara,E
23,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Burundi,E


## Adding information to the linkset

Three possible approaches
1. Querying local dump (when possible)
2. crawing each URI via python script
    *  too much http request
3. crawling via LD_spider

    

#### 1. Querying ThIST on local dump 

In [47]:
#variable 
path='data/ToBeValidateRevised'

In [48]:
%%bash -s "$path" 
less $1/GenerateCSV4ValidationOnLocalDumps.sparql

prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix skosxl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX  :       <.>

select distinct 
(GROUP_CONCAT(?sBroaderPrefLabel; separator="| ") as ?sBT) 
(GROUP_CONCAT(?sNarrowerPrefLabel; separator="| ") as ?sNT)
(GROUP_CONCAT(?sRelatedPrefLabel; separator="| ") as ?sRT) 
?s 
?sPrefLabel 
(GROUP_CONCAT(?sAltLabel ; separator="| ") as ?sAltLabels) ?o 
where {
?s skos:exactMatch ?o. 
 filter regex(str(?s),"http://linkeddata.ge.imati.cnr.it/resource/ThIST/",'i').
 filter regex(str(?o),"^http://dbpedia.org/",'i')  
   optional { ?s skos:prefLabel ?sPrefLabel. filter (lang(?sPrefLabel)='en')}. 
    optional { ?s skos:altLabel  ?sAltLabel . filter (lang(?sAltLabel)='en')}. 
   optional{
     ?s skos:broader ?sBroader. ?sBroader skos:prefLabel ?sBroaderPrefLabel. filter (lang(?sBroaderPrefLabel)='en') } . 
   optional{
     ?s skos:narrower ?sNarrower. ?sNarrower skos:prefLabel ?sNarrowerPrefLabel. filter (lang(?sNarrowerPrefLabel)='en') } . 
 

In [49]:
%%bash -s "$path" 
java arq.sparql  --data $1/THIST2DBPEDIANEW.nt --data $1/ThIST20140804.ttl \
 --query $1/GenerateCSV4ValidationOnLocalDumps.sparql --results CSV  > $1/THISTTHIST2DBPEDIANEWsub.csv


In [50]:
import pandas as pd

# reading Thist2DBPEDIA_ok.csv
THIST_dfa=pd.read_csv(path+'/THISTTHIST2DBPEDIANEWsub.csv',delimiter=",")
#THIST_dfa.head(2)
THIST_df=THIST_dfa.fillna("")
THIST_df.head()
#THIST_df.head()
#print(THIST_df.shape[0])

#for index, row in THIST_df.iterrows():
#        print (row['sBT'])


,sBT,sNT,sRT,s,sPrefLabel,sAltLabels,o
0,,,diamonds| industrial minerals| corundum deposi...,http://linkeddata.ge.imati.cnr.it/resource/ThI...,abrasives,,http://dbpedia.org/resource/Category:Abrasives
1,seismology| seismology| seismology| seismology...,,accelerometers| motions| seismic response| vel...,http://linkeddata.ge.imati.cnr.it/resource/ThI...,acceleration,,http://dbpedia.org/resource/Category:Acceleration
2,,,steel industry| metallurgy| alloys| iron,http://linkeddata.ge.imati.cnr.it/resource/ThI...,steel,,http://dbpedia.org/resource/Category:Steel
3,,sulfuric acid| sulfuric acid| sulfuric acid| s...,acidic composition| pH| DNA| acidity| buffers|...,http://linkeddata.ge.imati.cnr.it/resource/ThI...,acids,,http://dbpedia.org/resource/Category:Acids
4,,ground water| ground water| ground water| grou...,percolation| retention| oil-water interface| m...,http://linkeddata.ge.imati.cnr.it/resource/ThI...,water,,http://dbpedia.org/resource/Category:Water


In [51]:

#claning duplicates on sBT cells
#http://jonathansoma.com/lede/foundations/classes/pandas%20columns%20and%20functions/apply-a-function-to-every-row-in-a-pandas-dataframe/
def deletingDuplicates( df, columnName):
    for index, row in df.iterrows():
    #    print (row[columnName])
        # Let iterate for every row in the DF
        a= map(str.strip, df.iloc[index][columnName].split('|'))
        mylist = list(dict.fromkeys(a))
        s='| '.join(str(f) for f in mylist)
        df.at[index,columnName]=s
    return df

NEWTHIST=deletingDuplicates(THIST_df,'sBT')
NEWTHIST=deletingDuplicates(NEWTHIST,'sNT')
NEWTHIST=deletingDuplicates(NEWTHIST,'sRT')
NEWTHIST=deletingDuplicates(NEWTHIST,'sAltLabels')

NEWTHIST.to_csv(path+'/THISTTHIST2DBPEDIANEWsubCLean.csv', sep=',', index=False)
NEWTHIST.head(20)

,sBT,sNT,sRT,s,sPrefLabel,sAltLabels,o
0,,,diamonds| industrial minerals| corundum deposi...,http://linkeddata.ge.imati.cnr.it/resource/ThI...,abrasives,,http://dbpedia.org/resource/Category:Abrasives
1,seismology,,accelerometers| motions| seismic response| vel...,http://linkeddata.ge.imati.cnr.it/resource/ThI...,acceleration,,http://dbpedia.org/resource/Category:Acceleration
2,,,steel industry| metallurgy| alloys| iron,http://linkeddata.ge.imati.cnr.it/resource/ThI...,steel,,http://dbpedia.org/resource/Category:Steel
3,,sulfuric acid| humic acids| inorganic acids| h...,acidic composition| pH| DNA| acidity| buffers|...,http://linkeddata.ge.imati.cnr.it/resource/ThI...,acids,,http://dbpedia.org/resource/Category:Acids
4,,ground water,percolation| retention| oil-water interface| m...,http://linkeddata.ge.imati.cnr.it/resource/ThI...,water,,http://dbpedia.org/resource/Category:Water
5,,confined aquifers| alluvium aquifers| aquiclud...,hydrogeology| groundwater level| artesian well...,http://linkeddata.ge.imati.cnr.it/resource/ThI...,aquifers,,http://dbpedia.org/resource/Category:Aquifers
6,Zoantharia| Coelenterata| Invertebrata| Anthozoa,,,http://linkeddata.ge.imati.cnr.it/resource/ThI...,Actiniaria,,http://dbpedia.org/resource/Category:Actiniaria
7,,,applied geology| environmental geology| land use,http://linkeddata.ge.imati.cnr.it/resource/ThI...,airfields,,http://dbpedia.org/resource/Category:Airfields
8,,,meteorology| radioactive tracers| convection| ...,http://linkeddata.ge.imati.cnr.it/resource/ThI...,aerosols,,http://dbpedia.org/resource/Category:Aerosols
9,Middle East| Asia,Badakhshan Afghanistan| Kabul Afghanistan,Hindu Kush| Turkestan| Pamirs,http://linkeddata.ge.imati.cnr.it/resource/ThI...,Afghanistan,,http://dbpedia.org/resource/Category:Afghanistan


In [52]:
# TODO merge THIST2DBPEDIA_df_linkeset with NEWTHIST
import pandas as pd
NEWTHIST=pd.read_csv(path+'/THISTTHIST2DBPEDIANEWsubCLean.csv',delimiter=",")

NEWTHIST=NEWTHIST.fillna("")
#print(NEWTHIST)
THIST2DBPEDIA_df_linkeset=THIST2DBPEDIA_df_linkeset.fillna("")
#print(THIST2DBPEDIA_df_linkeset)
THIST2DBPEDIA_df_linkeset.describe()

result= pd.merge(THIST2DBPEDIA_df_linkeset, NEWTHIST, on=['o','s'])

result.head()

#Keys=NEWTHIST[['s','o']]
#finding duplicates
#https://thispointer.com/pandas-find-duplicate-rows-in-a-dataframe-based-on-all-or-selected-columns-using-dataframe-duplicated-in-python/
#duplicateRowsINNEWTHIST = Keys[Keys.duplicated()]
#NEWTHIST=NEWTHIST.drop_duplicates(subset=['s','o'], keep='first')
#THIST2DBPEDIA_df_linkeset=THIST2DBPEDIA_df_linkeset.drop_duplicates(subset=['s','o'], keep='first')
#print("Duplicate Rows except first occurrence based on all columns are :")
#print(duplicateRowsDF)
#print(duplicateRowsDF.count())
#print(THIST2DBPEDIA_df_linkeset.iloc[0]) #.head()
#print(NEWTHIST.iloc[0])
#result = pdL.merge(NEWTHIST, THIST2DBPEDIA_df_linkeset, on=['s','o'])
#NEWTHIST.describe()

#THIST2DBPEDIA_df_linkeset.head()
#Keys=THIST2DBPEDIA_df_linkeset[['s','o']]
#duplicateRowsINTHIST2DBPEDIA_df_linkeset = Keys[Keys.duplicated()]

#print(NEWTHIST)
#df4.DataFrame.duplicated(subset=['s','o'], keep='first')
#df4 = NEWTHIST.drop_duplicates(subset=['s','o'], keep='first')

#df5= THIST2DBPEDIA_df_linkeset.drop_duplicates(subset=['s','o'], keep='first')
#print(NEWTHIST.count()-duplicateRowsDF.count())
#print(df4.count())
#print(df4.count())
#df4.to_csv(path+'/df4.csv', sep=',', index=False)
#result = pd.merge(df4, df5, on=['s','o'], how='outer', validate="many_to_many")


,s,o,KindOfLink,sBT,sNT,sRT,sPrefLabel,sAltLabels
0,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Madagascar,E,Africa,Antananarivo,Indian Ocean| West Indian Ocean| Mozambique Ch...,Madagascar,
1,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Niger,E,Africa,Niamey Niger,Niger River Basin| Niger River| Sahel| Chad Ba...,Niger,
2,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Nubia,E,Africa,,Egypt| Sudan| Nile Valley,Nubia,
3,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Sahara,E,Africa,Western Sahara,Algeria| Algeri Algeria| Libya| Libyan Desert|...,Sahara,
4,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Burundi,E,Africa| Central Africa,Bujumbura Burundi,East African Rift| Lake Tanganyika| Nile River...,Burundi,


#### 2. Querying DBPEDIA 

We consider the DBPedia dumps created in 
* [Crawling RDF for DBPEDIA Objects In THIST Links](http://localhost:8888/notebooks/LinkCorrectess/CrawlingRDFforDBPEDIAObjectsInTHISTLinks.ipynb)

In [53]:
%%bash -s "$path" 
less $1/GenerateCSV4ValidationOnLocalLinkesetAndService.sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

select distinct 
?o 
?oPrefLabel 
?oDefinition
(GROUP_CONCAT(?oAltLabel ; separator="| ") as ?oAltLabels) 
(GROUP_CONCAT(?oBroaderPrefLabel; separator="| ") as ?oBT) 
(GROUP_CONCAT(?oNarrowerPrefLabel; separator="| ") as ?oNT)
(GROUP_CONCAT(?oRelatedPrefLabel; separator="| ") as ?oRT) 

where {
?s skos:exactMatch ?o. 
 filter regex(str(?s),"http://linkeddata.ge.imati.cnr.it/resource/ThIST/",'i').
 filter regex(str(?o),"^http://dbpedia.org/",'i')  
 optional { ?o skos:prefLabel ?oPrefLabel. filter (lang(?oPrefLabel)='en')}. 
   optional { ?o skos:definition ?oDefinition . filter (lang(?oDefinition)='en')}. 
   optional { ?o skos:altLabel  ?oAltLabel . filter (lang(?oAltLabel)='en')}. 
   optional{
     ?o skos:broader ?oBroader. ?oBroader skos:prefLabel ?oBroaderPrefLabel. filter (lang(?oBroaderPrefLabel)='en') } . 
   option

In [54]:
%%bash -s "$path" 
java -Xmx6072M arq.sparql --data $1/THIST2DBPEDIANEW.nt --data data/crawling/DBPEDIA_Crawling.ttl \
 --query $1/GenerateCSV4ValidationOnLocalLinkesetAndService.sparql --results CSV  > $1/DBPEDIATHIST2DBPEDIANEWsub.csv


WARN  [line: 70156, col: 292] Bad IRI: <http://ja.dbpedia.org/resource/Category:メクレンブルク＝フォアポンメルン> Code: 47/NOT_NFKC in PATH: The IRI is not in Unicode Normal Form KC.
WARN  [line: 70156, col: 292] Bad IRI: <http://ja.dbpedia.org/resource/Category:メクレンブルク＝フォアポンメルン> Code: 56/COMPATIBILITY_CHARACTER in PATH: Bad character
WARN  [line: 81103, col: 1 ] Bad IRI: <http://dbpedia.org/resource/A¹_homotopy_theory> Code: 47/NOT_NFKC in PATH: The IRI is not in Unicode Normal Form KC.
WARN  [line: 81103, col: 1 ] Bad IRI: <http://dbpedia.org/resource/A¹_homotopy_theory> Code: 56/COMPATIBILITY_CHARACTER in PATH: Bad character
WARN  [line: 132195, col: 1 ] Bad IRI: <http://dbpedia.org/resource/L²_cohomology> Code: 47/NOT_NFKC in PATH: The IRI is not in Unicode Normal Form KC.
WARN  [line: 132195, col: 1 ] Bad IRI: <http://dbpedia.org/resource/L²_cohomology> Code: 56/COMPATIBILITY_CHARACTER in PATH: Bad character
WARN  [line: 266630, col: 806] Bad IRI: <http://ja.dbpedia.org/resource/Category:ブーシュ＝デュ＝

In [55]:
# TODO merge THIST2DBPEDIA_df_linkeset with NEWTHIST
import pandas as pdL 
DBpedia=pd.read_csv(path+'/DBPEDIATHIST2DBPEDIANEWsub.csv',delimiter=",")

DBpedia=DBpedia.fillna("")
#print(NEWTHIST)
NEWDBpedia=deletingDuplicates(DBpedia,'oBT')
NEWDBpedia=deletingDuplicates(NEWDBpedia,'oNT')
NEWDBpedia=deletingDuplicates(NEWDBpedia,'oRT')
NEWDBpedia=deletingDuplicates(NEWDBpedia,'oAltLabels')



NEWDBpedia.to_csv(path+'/NEWDBpediasubCLean.csv', sep=',', index=False)
NEWDBpedia.head(20)



,o,oPrefLabel,oDefinition,oAltLabels,oBT,oNT,oRT
0,http://dbpedia.org/resource/Category:Abrasives,Abrasives,,,Manufacturing| Grinding and lapping,,
1,http://dbpedia.org/resource/Category:Acceleration,Acceleration,,,Length| Physical quantities| Dynamics (mechani...,,
2,http://dbpedia.org/resource/Category:Acids,Acids,,,Acid–base chemistry| Chemical compounds,,
3,http://dbpedia.org/resource/Category:Actiniaria,Actiniaria,,,Hexacorallia,,
4,http://dbpedia.org/resource/Category:Actinides,Actinides,,,Sets of chemical elements| Metallic elements,,
5,http://dbpedia.org/resource/Category:Actinium,Actinium,,,Actinides| Chemical elements| Wikipedia catego...,,
6,http://dbpedia.org/resource/Category:Aerosols,Aerosols,,,Colloids,,
7,http://dbpedia.org/resource/Category:Afghanistan,Afghanistan,,,Pashto-speaking countries and territories| Ira...,,
8,http://dbpedia.org/resource/Category:Africa,Africa,,,Wikipedia categories named after continents| A...,,
9,http://dbpedia.org/resource/Category:Age,Age,,,,,


In [57]:
# TODO merge THIST2DBPEDIA_df_linkeset with NewDBPEDIA
import pandas as pd 
path="data/" # path where to find data

NEWDBpedia=NEWDBpedia.fillna("")
#print(NEWTHIST)
THIST2DBPEDIA_df_linkeset=THIST2DBPEDIA_df_linkeset.fillna("")
#print(THIST2DBPEDIA_df_linkeset)

result1=pd.merge(result, NEWDBpedia, on=['o'])
result1.to_csv(path+'/ThIST2BpediaEnrichedLinkeset.csv', sep=',')

